1. Download and explore the data
2. Prepare the data for training
2. Build a recurrent neural network
3. Train & evaluate the model

## Import dataset

In [ ]:
# Download a specific dataset
!kaggle datasets download -d julian3833/jigsaw-toxic-comment-classification-challenge

# Optionally, unzip and read data
import pandas as pd
import zipfile

# Unzip if necessary
with zipfile.ZipFile("/content/jigsaw-toxic-comment-classification-challenge.zip", 'r') as zip_ref:
    zip_ref.extractall()

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


FileNotFoundError: [Errno 2] No such file or directory: '/content/jigsaw-toxic-comment-classification-challenge.zip'

In [9]:
# read csv data
import pandas as pd  # Import the pandas library
# Assuming the files are in the current working directory after extraction:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
#working with 50000 data
train_df = train_df.iloc[:50000]

In [11]:
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [12]:
train_df.shape

(50000, 8)

In [13]:
train_df.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
34781,5ce1897d69f89c6f,"""\n\n2005 rule changes """"aimed at Ferrari""""\n""...",0,0,0,0,0,0
17543,2e5f823326a84ccd,"""\n\nI am 0101NGhk. In fact, though it is good...",0,0,0,0,0,0
11892,1f7efa23140d33f5,"That's so cool that you like the Red Sox, they...",0,0,0,0,0,0
38570,66eb413f6379af56,Category:Hotels in Maine \n\nI undid you chang...,0,0,0,0,0,0
28897,4c9073e5bd592928,"From memory, it seemed to work okay on my rela...",0,0,0,0,0,0


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             50000 non-null  object
 1   comment_text   50000 non-null  object
 2   toxic          50000 non-null  int64 
 3   severe_toxic   50000 non-null  int64 
 4   obscene        50000 non-null  int64 
 5   threat         50000 non-null  int64 
 6   insult         50000 non-null  int64 
 7   identity_hate  50000 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 3.1+ MB


In [7]:
train_df.comment_text.values[1]

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [8]:
target_col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [9]:
# Getting all comments categories 0 and 1 ratio
for col in target_col:
    print(train_df[col].value_counts(normalize=True))

toxic
0    0.90234
1    0.09766
Name: proportion, dtype: float64
severe_toxic
0    0.9897
1    0.0103
Name: proportion, dtype: float64
obscene
0    0.9471
1    0.0529
Name: proportion, dtype: float64
threat
0    0.99664
1    0.00336
Name: proportion, dtype: float64
insult
0    0.95102
1    0.04898
Name: proportion, dtype: float64
identity_hate
0    0.99114
1    0.00886
Name: proportion, dtype: float64


In [10]:
y = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

## Prepare Dataset for Training


*   Convert text to TF-IDF vectors
*   Split training & validation set

In [11]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional , SimpleRNN , Dropout , BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

In [16]:
#Clean the text data to remove unnecessary characters, punctuation, and normalize the text.
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK resources (run once if needed)
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab') # Add this line to download the missing resource

# Function to clean and preprocess text
def preprocess_text(text):
    # Remove non-alphanumeric characters and multiple spaces
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize words
    tokens = word_tokenize(text)

    # Remove stopwords and lemmatize tokens
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and not word.isnumeric()]

    # Join tokens back into text
    processed_text = ' '.join(filtered_tokens)

    return processed_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [17]:
# If you want to reset the index to start from 0, use:
train_df = train_df.reset_index(drop=True)
sample_text = train_df['comment_text'][0]
print(sample_text)

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


In [18]:
preprocess_text(sample_text)

'explanation edits made username hardcore metallica fan reverted vandalism closure gas voted new york doll fac please remove template talk page since retired'

In [19]:
# Apply preprocessing function to 'comment_text' column
train_df['clean_comment'] = train_df['comment_text'].apply(preprocess_text)

In [20]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_comment
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation edits made username hardcore metal...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,aww match background colour seemingly stuck th...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man really trying edit war guy constantly ...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,make real suggestion improvement wondered sect...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,sir hero chance remember page


In [21]:
# Applying TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features= 1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(train_df['clean_comment'])
print(tfidf_matrix.shape)

(50000, 1000)


In [22]:
from collections import Counter
print('toxic',sorted(Counter(train_df['toxic']).items()))
print('severe_toxic',sorted(Counter(train_df['severe_toxic']).items()))
print('obscene',sorted(Counter(train_df['obscene']).items()))
print('threat',sorted(Counter(train_df['threat']).items()))
print('insult',sorted(Counter(train_df['insult']).items()))
print('identity_hate',sorted(Counter(train_df['identity_hate']).items()))

toxic [(0, 45117), (1, 4883)]
severe_toxic [(0, 49485), (1, 515)]
obscene [(0, 47355), (1, 2645)]
threat [(0, 49832), (1, 168)]
insult [(0, 47551), (1, 2449)]
identity_hate [(0, 49557), (1, 443)]


In [8]:
#split the data
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(tfidf_matrix, y, test_size=0.3, random_state=42)

NameError: name 'tfidf_matrix' is not defined

In [24]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)


X_train shape: (35000, 1000)
X_val shape: (15000, 1000)
y_train shape: (35000, 6)
y_val shape: (15000, 6)


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Assuming `y_train` contains the labels and `tfidf_matrix` is your TF-IDF matrix

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(tfidf_matrix, y, test_size=0.2, random_state=42)

# Initialize the One-vs-Rest Logistic Regression model
model = OneVsRestClassifier(LogisticRegression(max_iter=1000, class_weight='balanced'))

# Train the model
model.fit(X_train, y_train)

# Predict on the validation set
y_pred = model.predict(X_val)

# Evaluate the model
print(classification_report(y_val, y_pred, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))


               precision    recall  f1-score   support

        toxic       0.47      0.79      0.59      1042
 severe_toxic       0.18      0.89      0.30       101
      obscene       0.46      0.86      0.60       539
       threat       0.08      0.79      0.15        29
       insult       0.37      0.82      0.51       493
identity_hate       0.11      0.78      0.20        77

    micro avg       0.36      0.82      0.50      2281
    macro avg       0.28      0.82      0.39      2281
 weighted avg       0.42      0.82      0.54      2281
  samples avg       0.05      0.08      0.06      2281



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
from sklearn.metrics import accuracy_score

# Predict on the validation set
y_pred = model.predict(X_val)

# Calculate accuracy for each sample
sample_accuracies = [accuracy_score(y_val[i], y_pred[i]) for i in range(len(y_val))]

# Calculate overall accuracy
overall_accuracy = np.mean(sample_accuracies)

print(f'Overall Accuracy: {overall_accuracy}')


Overall Accuracy: 0.9382333333333333


### Decision Tree

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np

# Initialize the One-vs-Rest Decision Tree model
decision_tree_model = OneVsRestClassifier(DecisionTreeClassifier(class_weight='balanced'))

# Train the model
decision_tree_model.fit(X_train, y_train)

# Predict on the validation set
y_pred_dt = decision_tree_model.predict(X_val)

# Evaluate the model
print("Decision Tree Classification Report:")
print(classification_report(y_val, y_pred_dt, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

# Calculate accuracy for each sample
sample_accuracies_dt = [accuracy_score(y_val[i], y_pred_dt[i]) for i in range(len(y_val))]

# Calculate overall accuracy
overall_accuracy_dt = np.mean(sample_accuracies_dt)
print(f'Decision Tree Overall Accuracy: {overall_accuracy_dt}')


Decision Tree Classification Report:
               precision    recall  f1-score   support

        toxic       0.45      0.67      0.54      1042
 severe_toxic       0.11      0.48      0.18       101
      obscene       0.48      0.77      0.59       539
       threat       0.14      0.55      0.22        29
       insult       0.34      0.61      0.44       493
identity_hate       0.09      0.47      0.15        77

    micro avg       0.35      0.66      0.46      2281
    macro avg       0.27      0.59      0.35      2281
 weighted avg       0.40      0.66      0.49      2281
  samples avg       0.06      0.06      0.06      2281



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Decision Tree Overall Accuracy: 0.9412166666666668


## Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np

# Initialize the One-vs-Rest Random Forest model
random_forest_model = OneVsRestClassifier(RandomForestClassifier(class_weight='balanced'))

# Train the model
random_forest_model.fit(X_train, y_train)

# Predict on the validation set
y_pred_rf = random_forest_model.predict(X_val)

# Evaluate the model
print("Random Forest Classification Report:")
print(classification_report(y_val, y_pred_rf, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

# Calculate accuracy for each sample
sample_accuracies_rf = [accuracy_score(y_val[i], y_pred_rf[i]) for i in range(len(y_val))]

# Calculate overall accuracy
overall_accuracy_rf = np.mean(sample_accuracies_rf)
print(f'Random Forest Overall Accuracy: {overall_accuracy_rf}')


Random Forest Classification Report:
               precision    recall  f1-score   support

        toxic       0.68      0.58      0.62      1042
 severe_toxic       0.08      0.21      0.12       101
      obscene       0.60      0.71      0.65       539
       threat       0.36      0.17      0.23        29
       insult       0.48      0.58      0.53       493
identity_hate       0.10      0.29      0.15        77

    micro avg       0.51      0.58      0.54      2281
    macro avg       0.38      0.42      0.38      2281
 weighted avg       0.57      0.58      0.56      2281
  samples avg       0.05      0.05      0.05      2281



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Random Forest Overall Accuracy: 0.96245


In [29]:
!pip install imbalanced-learn

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score
import scipy.sparse as sp

def manual_balance(X, y):
    df_X = pd.DataFrame(X)
    df_y = pd.DataFrame(y, columns=[f'label_{i}' for i in range(y.shape[1])])

    # Track indices for resampling
    balanced_indices = []

    for label in df_y.columns:
        # Find minority and majority class indices
        minority_indices = df_y[df_y[label] == 1].index
        majority_indices = df_y[df_y[label] == 0].index

        # Calculate the number of samples needed to balance
        num_to_duplicate = len(majority_indices) - len(minority_indices)

        if num_to_duplicate > 0:
            # Duplicate minority samples to balance
            duplicated_indices = np.random.choice(minority_indices, num_to_duplicate, replace=True)
            resampled_indices = np.concatenate([minority_indices, duplicated_indices])
        else:
            resampled_indices = minority_indices

        balanced_indices.extend(resampled_indices)

    # Drop duplicates and create balanced dataset
    balanced_indices = list(set(balanced_indices))
    balanced_df_X = df_X.loc[balanced_indices]
    balanced_df_y = df_y.loc[balanced_indices]

    return balanced_df_X.values, balanced_df_y.values

# Convert TF-IDF matrix to dense format (if sparse)
if sp.issparse(tfidf_matrix):
    tfidf_matrix = tfidf_matrix.toarray()

# Apply manual balancing
X_balanced, y_balanced = manual_balance(tfidf_matrix, y)

# Split the balanced data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Initialize the One-vs-Rest Logistic Regression model
model = OneVsRestClassifier(LogisticRegression(max_iter=1000, class_weight='balanced'))

# Train the model
model.fit(X_train, y_train)

# Predict on the validation set
y_pred = model.predict(X_val)

# Evaluate the model
print("Logistic Regression Classification Report after Manual Balancing:")
print(classification_report(y_val, y_pred, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

# Calculate accuracy for each sample
sample_accuracies = [accuracy_score(y_val[i], y_pred[i]) for i in range(len(y_val))]

# Calculate overall accuracy
overall_accuracy = np.mean(sample_accuracies)
print(f'Overall Accuracy after Manual Balancing: {overall_accuracy}')


Logistic Regression Classification Report after Manual Balancing:
               precision    recall  f1-score   support

        toxic       0.96      0.82      0.89       985
 severe_toxic       0.33      0.73      0.46       114
      obscene       0.88      0.71      0.79       511
       threat       0.22      0.61      0.33        28
       insult       0.72      0.64      0.68       481
identity_hate       0.37      0.66      0.48        93

    micro avg       0.76      0.74      0.75      2212
    macro avg       0.58      0.69      0.60      2212
 weighted avg       0.82      0.74      0.77      2212
  samples avg       0.67      0.71      0.66      2212



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Overall Accuracy after Manual Balancing: 0.8216941480763014


In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize the One-vs-Rest Decision Tree model
decision_tree_model = OneVsRestClassifier(DecisionTreeClassifier())

# Train the model
decision_tree_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = decision_tree_model.predict(X_val)

# Evaluate the model
print("Decision Tree Classification Report after Manual Balancing:")
print(classification_report(y_val, y_pred, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

# Calculate accuracy for each sample
sample_accuracies = [accuracy_score(y_val[i], y_pred[i]) for i in range(len(y_val))]

# Calculate overall accuracy
overall_accuracy = np.mean(sample_accuracies)
print(f'Overall Accuracy after Manual Balancing: {overall_accuracy}')


Decision Tree Classification Report after Manual Balancing:
               precision    recall  f1-score   support

        toxic       0.95      0.96      0.96       985
 severe_toxic       0.33      0.20      0.25       114
      obscene       0.73      0.74      0.73       511
       threat       0.15      0.14      0.15        28
       insult       0.60      0.64      0.62       481
identity_hate       0.43      0.40      0.41        93

    micro avg       0.77      0.77      0.77      2212
    macro avg       0.53      0.51      0.52      2212
 weighted avg       0.76      0.77      0.76      2212
  samples avg       0.79      0.81      0.75      2212



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Overall Accuracy after Manual Balancing: 0.8347882314904624


In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize the One-vs-Rest Random Forest model
random_forest_model = OneVsRestClassifier(RandomForestClassifier(n_estimators=100, n_jobs=-1))

# Train the model
random_forest_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = random_forest_model.predict(X_val)

# Evaluate the model
print("Random Forest Classification Report after Manual Balancing:")
print(classification_report(y_val, y_pred, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

# Calculate accuracy for each sample
sample_accuracies = [accuracy_score(y_val[i], y_pred[i]) for i in range(len(y_val))]

# Calculate overall accuracy
overall_accuracy = np.mean(sample_accuracies)
print(f'Overall Accuracy after Manual Balancing: {overall_accuracy}')


Random Forest Classification Report after Manual Balancing:
               precision    recall  f1-score   support

        toxic       0.96      1.00      0.98       985
 severe_toxic       0.46      0.05      0.09       114
      obscene       0.81      0.75      0.78       511
       threat       0.56      0.18      0.27        28
       insult       0.67      0.67      0.67       481
identity_hate       0.54      0.27      0.36        93

    micro avg       0.84      0.78      0.81      2212
    macro avg       0.67      0.49      0.53      2212
 weighted avg       0.81      0.78      0.78      2212
  samples avg       0.86      0.83      0.81      2212

Overall Accuracy after Manual Balancing: 0.8687358551568058


In [33]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Initialize the One-vs-Rest SVM model
svm_model = OneVsRestClassifier(SVC(kernel='linear', probability=True))

# Train the model
svm_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = svm_model.predict(X_val)

# Evaluate the model
print("SVM Classification Report after Manual Balancing:")
print(classification_report(y_val, y_pred, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

# Calculate accuracy for each sample
sample_accuracies = [accuracy_score(y_val[i], y_pred[i]) for i in range(len(y_val))]

# Calculate overall accuracy
overall_accuracy = np.mean(sample_accuracies)
print(f'Overall Accuracy after Manual Balancing: {overall_accuracy}')


SVM Classification Report after Manual Balancing:
               precision    recall  f1-score   support

        toxic       0.96      1.00      0.98       985
 severe_toxic       0.00      0.00      0.00       114
      obscene       0.87      0.71      0.78       511
       threat       0.64      0.25      0.36        28
       insult       0.73      0.62      0.67       481
identity_hate       0.65      0.33      0.44        93

    micro avg       0.88      0.76      0.82      2212
    macro avg       0.64      0.49      0.54      2212
 weighted avg       0.82      0.76      0.78      2212
  samples avg       0.89      0.81      0.82      2212



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Overall Accuracy after Manual Balancing: 0.8769802780472035


In [34]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.5 MB/s eta 0:00:00


In [35]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Define parameters
maxlen = 500  # Maximum length of sequences (adjust based on your dataset)

# Pad sequences to ensure they are of the same length
X_padded = pad_sequences(X_balanced, maxlen=maxlen, padding='post')

# Split the padded data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_padded, y_balanced, test_size=0.2, random_state=42)

# Define the RNN model with increased complexity
model = Sequential()
model.add(Embedding(input_dim=X_train.shape[1], output_dim=256, input_length=maxlen))  # Increased embedding dimension
model.add(SimpleRNN(units=128, return_sequences=True))  # Increased number of units and added return_sequences=True
model.add(Dropout(0.5))  # Added Dropout layer to prevent overfitting
model.add(SimpleRNN(units=64, return_sequences=False))  # Added another RNN layer
model.add(Dropout(0.5))  # Added Dropout layer
model.add(Dense(units=y_train.shape[1], activation='sigmoid'))  # Use sigmoid for multi-label classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor the validation loss
                               patience=3,  # Number of epochs with no improvement to wait before stopping
                               restore_best_weights=True)  # Restore model weights from the epoch with the best value of the monitored quantity

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=10,  # Increased maximum number of epochs
                    batch_size=64,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])  # Include the early stopping callback

# Predict on the validation set
y_pred = model.predict(X_val)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary

# Evaluate the model
print("RNN Classification Report with Early Stopping:")
print(classification_report(y_val, y_pred_binary, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

# Calculate accuracy for each sample
sample_accuracies = [accuracy_score(y_val[i], y_pred_binary[i]) for i in range(len(y_val))]

# Calculate overall accuracy
overall_accuracy = np.mean(sample_accuracies)
print(f'Overall Accuracy with Early Stopping: {overall_accuracy}')


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.7366 - loss: 0.5043 - val_accuracy: 0.9554 - val_loss: 0.3960
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 108s 897ms/step - accuracy: 0.9197 - loss: 0.4280 - val_accuracy: 0.9554 - val_loss: 0.3935
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 85s 974ms/step - accuracy: 0.8552 - loss: 0.4702 - val_accuracy: 0.9554 - val_loss: 0.4059
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 83s 998ms/step - accuracy: 0.9237 - loss: 0.4461 - val_accuracy: 0.9554 - val_loss: 0.4053
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 63s 975ms/step - accuracy: 0.9380 - loss: 0.4372 - val_accuracy: 0.9554 - val_loss: 0.3982
33/33 ━━━━━━━━━━━━━━━━━━━━ 8s 236ms/step
RNN Classification Report with Early Stopping:
               precision    recall  f1-score   support

        toxic       0.96      1.00      0.98       985
 severe_toxic       0.00      0.00      0.00       114
      obscene       0.50      1.00      0.66       511
       threat       0.00      0.00      0.00   

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Overall Accuracy with Early Stopping: 0.7930811509860977


In [7]:
X_train

NameError: name 'X_train' is not defined

In [36]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Define parameters
maxlen = 500  # Maximum length of sequences (adjust based on your dataset)

# Pad sequences to ensure they are of the same length
X_padded = pad_sequences(X_balanced, maxlen=maxlen, padding='post')

# Split the padded data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_padded, y_balanced, test_size=0.2, random_state=42)

# Define the LSTM model with increased complexity
model = Sequential()
model.add(Embedding(input_dim=X_train.shape[1], output_dim=256, input_length=maxlen))  # Increased embedding dimension
model.add(LSTM(units=128, return_sequences=True))  # LSTM layer with 128 units
model.add(Dropout(0.5))  # Added Dropout layer to prevent overfitting
model.add(LSTM(units=64, return_sequences=False))  # Second LSTM layer with 64 units
model.add(Dropout(0.5))  # Added Dropout layer
model.add(Dense(units=y_train.shape[1], activation='sigmoid'))  # Output layer with sigmoid activation for multi-label classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor the validation loss
                               patience=3,  # Number of epochs with no improvement to wait before stopping
                               restore_best_weights=True)  # Restore model weights from the epoch with the best value of the monitored quantity

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=10,  # Increased maximum number of epochs
                    batch_size=64,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])  # Include the early stopping callback

# Predict on the validation set
y_pred = model.predict(X_val)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary

# Evaluate the model
print("LSTM Classification Report with Early Stopping:")
print(classification_report(y_val, y_pred_binary, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

# Calculate accuracy for each sample
sample_accuracies = [accuracy_score(y_val[i], y_pred_binary[i]) for i in range(len(y_val))]

# Calculate overall accuracy
overall_accuracy = np.mean(sample_accuracies)
print(f'Overall Accuracy with Early Stopping: {overall_accuracy}')


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


65/65 ━━━━━━━━━━━━━━━━━━━━ 170s 3s/step - accuracy: 0.8551 - loss: 0.4787 - val_accuracy: 0.9554 - val_loss: 0.3931
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 208s 3s/step - accuracy: 0.9452 - loss: 0.4069 - val_accuracy: 0.9554 - val_loss: 0.3918
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 210s 3s/step - accuracy: 0.9472 - loss: 0.4082 - val_accuracy: 0.9554 - val_loss: 0.3920
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 188s 3s/step - accuracy: 0.9504 - loss: 0.4008 - val_accuracy: 0.9554 - val_loss: 0.3913
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 160s 2s/step - accuracy: 0.9440 - loss: 0.4015 - val_accuracy: 0.9554 - val_loss: 0.3911
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.9415 - loss: 0.4023 - val_accuracy: 0.9554 - val_loss: 0.3912
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.9416 - loss: 0.4029 - val_accuracy: 0.9554 - val_loss: 0.3908
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.9448 - loss: 0.4005 - val_accuracy: 0.9554 - val_loss: 0.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Overall Accuracy with Early Stopping: 0.7927578402845135


In [6]:
import numpy as np
import gensim
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

# ------------------------
# 1. Prepare Data
# ------------------------
# Assuming you already have X_train, X_test, y_train, y_test
max_words = 20000
maxlen = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

# ------------------------
# 2. Load Word2Vec embeddings
# ------------------------
embedding_dim = 300
word_index = tokenizer.word_index

# Load pre-trained Google News Word2Vec (download manually)
w2v_path = 'GoogleNews-vectors-negative300.bin'
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(w2v_path, binary=True)

# Create embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in w2v_model:
        embedding_matrix[i] = w2v_model[word]

# ------------------------
# 3. Build LSTM Model
# ------------------------
model = Sequential()
model.add(Embedding(input_dim=len(word_index)+1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False))  # freeze embeddings
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y_train.shape[1], activation='sigmoid'))  # multi-label
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# ------------------------
# 4. Train Model
# ------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train_pad, y_train,
                    validation_split=0.2,
                    epochs=10,
                    batch_size=64,
                    callbacks=[early_stop],
                    verbose=1)

# ------------------------
# 5. Evaluate Accuracy
# ------------------------
y_pred = (model.predict(X_test_pad) > 0.5).astype(int)

# For multi-label classification, strict accuracy
acc = accuracy_score(y_test, y_pred)
print(f"LSTM with Word2Vec Accuracy: {acc:.4f}")


NameError: name 'X_train' is not defined

In [2]:
!pip install gensim